# Factory Simulation with SimPy
## Scenario
A factory gets **raw material**. The raw material is processed/assembled into pieces by **Piece Assembly Machines**. Then, the pieces are assembled into models by the **Model Assembly Machines**. From time to time, the machines break.

When a machine breaks, the factory calls a repair center, which has other callers. When the factory's request reaches an operator, they decide if intervention is necessary. If no intervention is needed, the factory workers can repair the machine by themselves. If intervention is necessary, a repairman is called to the premises, who then repairs the machine.

### Math Reminders - Exponential & Lognormal Distributions
They can be found in `pharmacy_sim.ipynb`. 

### Math Reminder - Normal Distribution
Also called the **Gaussian Distribution** is one of the most commonly used distributions in statistics.

Its PDF is:

$$
f(x) = \frac{1}{\sqrt{2\pi\sigma^2}}\exp(-\frac{(x-\mu)^2}{2\sigma^2})
$$

In [116]:
import numpy as np
import simpy
import pandas as pd
import itertools
from distributions import (
    Exponential,
    Lognormal,
    Normal,
    Bernoulli
)

In [117]:
# Default Parameters

TRACE = False

N_OPERATORS = 5
N_REPAIRMEN = 3
CALL_RATE = 3
TRAVEL_SIGMA = 0.04
TRAVEL_MU = 2.7
CALL_LENGTH_SIGMA = 0.1
CALL_LENGTH_MU = 1.7
REPAIR_TIME_MEAN = 8.5
REPAIR_TIME_VAR = 0.7
INTERVENTION_CHANCE = 0.2  
N_STREAMS = 5
DEFAULT_RND_SET = 42
COLLECTION_TIME = 24 * 60
WARMUP_TIME = 1 * 60

In [118]:
class Experiment:

    def __init__(
        self,
        random_nr_set = DEFAULT_RND_SET,
        n_streams = N_STREAMS,
        n_operators = N_OPERATORS,
        n_repairmen = N_REPAIRMEN,
        call_rate = CALL_RATE,
        call_length_sigma = CALL_LENGTH_SIGMA,
        call_length_mu = CALL_LENGTH_MU,
        travel_sigma = TRAVEL_SIGMA,
        travel_mu = TRAVEL_MU,
        repair_time_mean = REPAIR_TIME_MEAN,
        repair_time_var = REPAIR_TIME_VAR,
        intervention_chance = INTERVENTION_CHANCE
    ):
        self.random_nr_set = random_nr_set
        self.n_streams = n_streams
        self.n_operators = n_operators
        self.n_repairmen = n_repairmen
        self.call_scale = 1 / call_rate
        self.call_length_sigma = call_length_sigma
        self.call_length_mu = call_length_mu
        self.travel_sigma = travel_sigma
        self.travel_mu = travel_mu
        self.repair_time_mean = repair_time_mean
        self.repair_time_var = repair_time_var
        self.intervention_chance = intervention_chance

        self.operators = None
        self.repairmen = None

        self.init_results_vars()
        self.init_sampling()


    def set_random_nr_set(self, random_nr_set):
        self.random_nr_set = random_nr_set
        self.init_sampling()

    
    def init_sampling(self):
        seed_sequence = np.random.SeedSequence(self.random_nr_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        self.arrival_dist = Exponential(
            self.call_scale, 
            self.seeds[0]
        )
        self.call_dist = Lognormal(
            self.call_length_mu,
            self.call_length_sigma,
            self.seeds[1]
        )
        self.travel_dist = Lognormal(
            self.travel_mu,
            self.travel_sigma,
            self.seeds[2]
        )
        self.repair_dist = Normal(
            self.repair_time_mean,
            self.repair_time_var,
            self.seeds[3]
        )
        self.intervention_dist = Bernoulli(
            self.intervention_chance,
            self.seeds[4]
        )

    def init_results_vars(self):
        self.results = {}
        self.results["call_waiting_times"] = []
        self.results["total_call_duration"] = 0.0
        self.results["repairman_dispatch_waiting_times"] = []
        self.results["total_repairman_usage"] = 0.0

In [119]:
def trace(msg):
    if TRACE:
        print(msg)

In [120]:
def intervention(id, env, args):
    trace(f"Caller #{id} waiting for repairman")
    start_dispatch_wait = env.now
    with args.repairmen.request() as req:
        yield req
        repairman_dispatch_time = env.now - start_dispatch_wait
        args.results["repairman_dispatch_waiting_times"].append(repairman_dispatch_time)
        start_occupy = env.now
        travel_duration = args.travel_dist.sample()
        trace(f"Repairman dispatched for caller #{id} at {env.now:.2f}")
        yield env.timeout(travel_duration)
        trace(f"Repairman arrived at caller #{id} at {env.now:.2f}")
        repair_duration = args.repair_dist.sample()
        yield env.timeout(repair_duration)
        end_occupy = env.now
        occupied = end_occupy - start_occupy
        if hasattr(args, "measurement_start") and hasattr(args, "measurement_end"):
            overlap_start = max(start_occupy, args.measurement_start)
            overlap_end = min(end_occupy, args.measurement_end)
            overlap = max(0.0, overlap_end - overlap_start)
        else:
            overlap = occupied
        args.results["total_repairman_usage"] += overlap
        trace(f"Repairman finished repairing caller #{id} at {env.now:.2f}")

In [121]:
def call_service(id, env, args):
    start_wait = env.now
    
    with args.operators.request() as req:
        yield req
        waiting_time = env.now - start_wait
        args.results["call_waiting_times"].append(waiting_time)
        trace(f"An operator answered call #{id} at {env.now:.2f}")
        start_service = env.now
        call_duration = args.call_dist.sample()
        yield env.timeout(call_duration)
        end_service = env.now
        occupied = end_service - start_service
        if hasattr(args, "measurement_start") and hasattr(args, "measurement_end"):
            overlap_start = max(start_service, args.measurement_start)
            overlap_end = min(end_service, args.measurement_end)
            overlap = max(0.0, overlap_end - overlap_start)
        else:
            overlap = occupied
        args.results["total_call_duration"] += overlap
        trace(f"Call #{id} ended at {env.now:.2f}. Waiting time was {waiting_time:.2f}")

        need_intervention = args.intervention_dist.sample()
        if need_intervention:
            env.process(intervention(id, env, args))

In [122]:
def generator(env, args):
    for caller_count in itertools.count(start=1):
        inter_call_time = args.arrival_dist.sample()
        yield env.timeout(inter_call_time)
        trace(f"A call at {env.now:.2f}")
        env.process(call_service(caller_count, env, args))

In [123]:
def warmup(warmup_time, env, args):
    yield env.timeout(warmup_time)
    trace(f"{env.now:.2f}: Warm Up Complete.")
    args.init_results_vars()

In [124]:
def single_run(
    experiment,
    rep = 0,
    warmup_time = WARMUP_TIME,
    collection_time = COLLECTION_TIME
):
    run_results = {}
    experiment.init_results_vars()
    experiment.set_random_nr_set(rep)
    env = simpy.Environment()
    experiment.operators = simpy.Resource(env, capacity=experiment.n_operators)
    experiment.repairmen = simpy.Resource(env, capacity=experiment.n_repairmen)
    experiment.measurement_start = warmup_time
    experiment.measurement_end = warmup_time + collection_time
    env.process(generator(env, experiment))
    env.process(warmup(warmup_time, env, experiment))
    env.run(until=warmup_time+collection_time)

    run_results["mean_call_waiting_time"] = np.mean(experiment.results["call_waiting_times"])
    run_results["operator_util"] = (experiment.results["total_call_duration"] / 
                                    (collection_time * experiment.n_operators)) * 100.0
    run_results["mean_repairmen_arrival_time"] = np.mean(experiment.results["repairman_dispatch_waiting_times"])
    run_results["total_repairmen_util"] = (experiment.results["total_repairman_usage"] /
                                           (collection_time * experiment.n_repairmen)) * 100.0
    
    return run_results

In [125]:
def multiple_reps(
        experiment,
        warmup_time = WARMUP_TIME,
        collection_time = COLLECTION_TIME,
        n_reps = 5
):
    results = [
        single_run(experiment, rep, warmup_time, collection_time)
        for rep in range(n_reps)
    ]

    df_results = pd.DataFrame(results)
    df_results.index = np.arange(1, (len(df_results)) + 1)
    df_results.index.name = "rep"
    return df_results

In [133]:
scenario = Experiment(n_repairmen=6, n_operators=8, call_rate=3.5, intervention_chance=0.17)
results = multiple_reps(scenario, n_reps=5)
results.describe().round(1).T

,count,mean,std,min,25%,50%,75%,max
mean_call_waiting_time,5.0,0.2,0.0,0.1,0.2,0.2,0.2,0.2
operator_util,5.0,74.5,0.7,73.4,74.4,74.8,75.0,75.1
mean_repairmen_arrival_time,5.0,75.0,28.2,49.2,50.0,65.8,101.6,108.2
total_repairmen_util,5.0,99.6,0.1,99.4,99.5,99.5,99.6,99.7
